In [289]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34209)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34209)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

Spark master launched!
Creating new Spark session, name: west0_ws_seg2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:48


In [291]:
# 각 day 28
start_date = "2022-06-08"
end_date = "2022-07-05"

ws_start_date = "2022-07-13"
ws_day_28 = "2022-08-09"

In [292]:
ws_gcoin = load_data_mart("pc", ws_start_date, ws_day_28, "gcoin_use")

In [293]:
ws_gcoin.where(col("event_name").like("%workshop%")).select(sum("paid_use"), sum(col("paid_use") + col("free_use"))).show(truncate=False)

+-------------+--------------------------+
|sum(paid_use)|sum((paid_use + free_use))|
+-------------+--------------------------+
|1874838675   |2341605700                |
+-------------+--------------------------+



In [288]:
sphynx.stop()

In [268]:
def get_workshop_rate(start_date, end_date):
    open_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date)
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date)
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date)
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date)
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken")
    
    workshop_user = (open_crate.groupBy("AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
        .unionByName(craft.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
        .unionByName(disassemble.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.groupBy("AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft")))

    return workshop_user

In [271]:
ws_user = get_workshop_rate(ws_start_date, ws_day_28)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46407)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46407)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [272]:
ws_user

DataFrame[AccountId: string, amount: bigint, action: string]

In [74]:
only_costume_gcoin_by_user = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user.csv"))
costume_and_wsus_gcoin_by_user = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user.csv"))
only_wsus_gcoin_by_user = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user.csv"))

In [75]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumnRenamed("total_gcoin", "total_use").withColumn("wsus_cnt", lit(0))

In [76]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("seg_group", lit("a"))
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("seg_group", lit("b"))
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("seg_group", lit("c"))

In [77]:
pre_seg = only_costume_gcoin_by_user.unionByName(costume_and_wsus_gcoin_by_user).unionByName(only_wsus_gcoin_by_user).withColumn("pre_seg", concat(col("seg_group"), col("group")))

In [49]:
only_costume_gcoin_by_user2 = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user2.csv"))
costume_and_wsus_gcoin_by_user2 = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user2.csv"))
only_wsus_gcoin_by_user2 = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user2.csv"))

In [50]:
only_costume_gcoin_by_user2 = only_costume_gcoin_by_user2.withColumnRenamed("group", "group2").withColumn("wsus_cnt", lit(0)).withColumn("seg_group2", lit("a"))
costume_and_wsus_gcoin_by_user2 = costume_and_wsus_gcoin_by_user2.withColumnRenamed("group", "group2").withColumn("seg_group2", lit("b"))
only_wsus_gcoin_by_user2 = only_wsus_gcoin_by_user2.withColumnRenamed("group", "group2").withColumn("seg_group2", lit("c")).withColumn("type", lit("wsus")).withColumn("chest_cnt", lit(0))

In [52]:
post_seg = only_costume_gcoin_by_user2.unionByName(costume_and_wsus_gcoin_by_user2).unionByName(only_wsus_gcoin_by_user2)

In [53]:
post_seg = post_seg.withColumnRenamed("paid_use", "paid_use2").withColumnRenamed("total_use", "total_use2").withColumnRenamed("wsus_cnt", "wsus_cnt2").withColumn("post_seg", concat(col("seg_group2"), col("group2")))

In [65]:
pre_seg.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- total_use: long (nullable = true)
 |-- paid_use: long (nullable = true)
 |-- group: string (nullable = true)
 |-- wsus_cnt: long (nullable = true)
 |-- seg_group: string (nullable = false)
 |-- pre_seg: string (nullable = true)



In [66]:
post_seg.printSchema()

root
 |-- type: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- group2: string (nullable = true)
 |-- paid_use2: long (nullable = true)
 |-- total_use2: long (nullable = true)
 |-- chest_cnt: long (nullable = true)
 |-- wsus_cnt2: long (nullable = true)
 |-- seg_group2: string (nullable = false)
 |-- post_seg: string (nullable = true)



In [68]:
df = pre_seg.join(post_seg, "account_id", "full_outer")
mysql.insert_table(df, "labs", "workshop_user_seg_df")

In [78]:
only_costume_gcoin_by_user3 = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user3.csv"))
costume_and_wsus_gcoin_by_user3 = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user3.csv"))
only_wsus_gcoin_by_user3 = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user3.csv"))

In [79]:
only_costume_gcoin_by_user3 = only_costume_gcoin_by_user3.withColumnRenamed("group", "group2").withColumn("wsus_cnt", lit(0)).withColumn("seg_group2", lit("a"))
costume_and_wsus_gcoin_by_user3 = costume_and_wsus_gcoin_by_user3.withColumnRenamed("group", "group2").withColumn("seg_group2", lit("b"))
only_wsus_gcoin_by_user3 = only_wsus_gcoin_by_user3.withColumnRenamed("group", "group2").withColumn("seg_group2", lit("c")).withColumn("type", lit("wsus")).withColumn("chest_cnt", lit(0))

In [80]:
post_seg = only_costume_gcoin_by_user3.unionByName(costume_and_wsus_gcoin_by_user3).unionByName(only_wsus_gcoin_by_user3)

In [83]:
post_seg = post_seg.withColumnRenamed("paid_use", "paid_use2").withColumnRenamed("total_use", "total_use2").withColumnRenamed("wsus_cnt", "wsus_cnt2").withColumn("post_seg", concat(col("seg_group2"), col("group2")))

In [84]:
df = pre_seg.join(post_seg, "account_id", "full_outer")

In [85]:
mysql.insert_table(df, "labs", "workshop_user_seg_df2")

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60280)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

## Day 28

### Costume = Costume & Workshop

In [90]:
only_costume_gcoin_by_user_day28 = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user_day28.csv")) \
    .withColumnRenamed("total_gcoin", "total_use").withColumn("wsus_cnt", lit(0)) \
    .withColumn("seg_group", lit("a"))
costume_and_wsus_gcoin_by_user_day28 = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user_day28.csv")) \
    .withColumn("seg_group", lit("b"))
only_wsus_gcoin_by_user_day28 = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user.csv")) \
    .withColumn("seg_group", lit("c"))

In [91]:
pre_seg28 = only_costume_gcoin_by_user_day28 \
    .unionByName(costume_and_wsus_gcoin_by_user_day28) \
    .unionByName(only_wsus_gcoin_by_user_day28) \
    .withColumn("pre_seg", concat(col("seg_group"), col("group")))

In [92]:
only_costume_gcoin_by_user2_day28 = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user2_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("wsus_cnt", lit(0)).withColumn("seg_group2", lit("a"))
costume_and_wsus_gcoin_by_user2_day28 = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user2_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("seg_group2", lit("b"))
only_wsus_gcoin_by_user2_day28 = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user2_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("seg_group2", lit("c")).withColumn("type", lit("wsus")).withColumn("chest_cnt", lit(0))

In [93]:
post_seg28 = only_costume_gcoin_by_user2_day28.unionByName(costume_and_wsus_gcoin_by_user2_day28).unionByName(only_wsus_gcoin_by_user2_day28) \
    .withColumnRenamed("paid_use", "paid_use2").withColumnRenamed("total_use", "total_use2") \
    .withColumnRenamed("wsus_cnt", "wsus_cnt2").withColumn("post_seg", concat(col("seg_group2"), col("group2")))

In [94]:
df = pre_seg28.join(post_seg28, "account_id", "full_outer")
mysql.insert_table(df, "labs", "workshop_user_seg_df_day28")

In [273]:
df1 = mysql.read_table(spark, "labs", "workshop_user_seg_df_day28")

In [274]:
df1_ws_user = ws_user.join(df1.withColumnRenamed("account_id", "AccountId"), "AccountId")

In [283]:
df1_user_cnt = df1.groupBy("seg_group", "group", "seg_group2", "group2").agg(countDistinct("account_id").alias("total_user_cnt")).toPandas()

In [279]:
df1_ws_user_by_action = df1_ws_user.groupBy("seg_group", "group", "seg_group2", "group2", "action").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

In [286]:
pd.merge(df1_user_cnt, df1_ws_user_by_action, on=["seg_group", "group", "seg_group2", "group2"], how="outer").to_csv("./df1_ws_user_by_action_and_total_user_cnt.csv", index=False)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42946)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [280]:
df1_ws_user_by_action.to_csv("./df1_ws_user_by_action.csv", index=False)

In [101]:
df1.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- total_use: long (nullable = true)
 |-- paid_use: long (nullable = true)
 |-- group: string (nullable = true)
 |-- wsus_cnt: long (nullable = true)
 |-- seg_group: string (nullable = true)
 |-- pre_seg: string (nullable = true)
 |-- type: string (nullable = true)
 |-- group2: string (nullable = true)
 |-- paid_use2: long (nullable = true)
 |-- total_use2: long (nullable = true)
 |-- chest_cnt: long (nullable = true)
 |-- wsus_cnt2: long (nullable = true)
 |-- seg_group2: string (nullable = true)
 |-- post_seg: string (nullable = true)



In [161]:
df1.show(truncate=False)

+----------------------------------------+---------+--------+-----+--------+---------+-------+--------+------+---------+----------+---------+---------+----------+--------+
|account_id                              |total_use|paid_use|group|wsus_cnt|seg_group|pre_seg|type    |group2|paid_use2|total_use2|chest_cnt|wsus_cnt2|seg_group2|post_seg|
+----------------------------------------+---------+--------+-----+--------+---------+-------+--------+------+---------+----------+---------+---------+----------+--------+
|account.0003fac446d14dafac335fdd29d5ba64|null     |null    |null |null    |null     |null   |workshop|A     |0        |100       |0        |0        |b         |bA      |
|account.0003fac446d14dafac335fdd29d5ba64|null     |null    |null |null    |null     |null   |wsus    |A     |0        |400       |0        |2        |b         |bA      |
|account.0004a9456c2444758567dd8b5650151d|null     |null    |null |null    |null     |null   |wsus    |A     |0        |400       |0        

In [241]:
wsus_and_ws = df1.where((col("seg_group") == "c") & (col("seg_group2") == "b"))

In [242]:
wsus_and_ws = wsus_and_ws.withColumn("wsus_use", when(col("type") == "wsus", col("total_use2")).otherwise(lit(0))) \
    .withColumn("wsus_paid", when(col("type") == "wsus", col("paid_use2")).otherwise(lit(0))) \
    .withColumn("workshop_use", when(col("type") == "workshop", col("total_use2")).otherwise(lit(0))) \
    .withColumn("workshop_paid", when(col("type") == "workshop", col("paid_use2")).otherwise(lit(0))) \
    .groupBy("account_id").agg(sum("chest_cnt").alias("chest_cnt"), sum("wsus_cnt2").alias("wsus_cnt"), sum("wsus_use").alias("wsus_use"), sum("wsus_paid").alias("wsus_paid"), sum("workshop_use").alias("workshop_use"), sum("workshop_paid").alias("workshop_paid")) \
    .withColumn("chest ratio", round(col("wsus_cnt")/col("chest_cnt")))

In [257]:
wsus_ws_df = wsus_and_ws.toPandas()

In [258]:
wsus_ws_df['group'] = np.where(wsus_ws_df['wsus_cnt']<=10, 'A',
                   np.where(wsus_ws_df['wsus_cnt']<=50, 'B',
                   np.where(wsus_ws_df['wsus_cnt']<=100, 'C', 
                   np.where(wsus_ws_df['wsus_cnt']<=800, 'D', 
                            'E'))))

In [259]:
wsus_ws_df.head()

,account_id,chest_cnt,wsus_cnt,wsus_use,wsus_paid,workshop_use,workshop_paid,chest ratio,group
0,account.03d1f364caf5450a89a674378cb50685,3,2,400,20,1140,500,1.0,A
1,account.049c8d20adc0493590e3825b1d6f25bd,2,71,12800,11800,480,390,36.0,C
2,account.0a5ec48e491648329117116dabf13147,1,3,600,100,180,180,3.0,A
3,account.0abb0c3f005b406ab9c198443057adae,11,30,5400,5000,2880,2400,3.0,B
4,account.0b558a0f8bcb46d8b0246c96efb03c87,0,3,600,0,0,0,NaN,A


In [261]:
wsus_ws_df.groupby("group", as_index=False)["account_id"].count()

,group,account_id
0,A,17656
1,B,7643
2,C,2524
3,D,2330
4,E,72


In [263]:
wsus_ws_df.groupby("group", as_index=False)["chest ratio"].mean()

,group,chest ratio
0,A,1.118558
1,B,5.239241
2,C,13.130009
3,D,28.488043
4,E,29.484375


In [264]:
wsus_ws_df.groupby("group", as_index=False)[["chest_cnt", "wsus_cnt", "workshop_use", "workshop_paid", "wsus_use", "wsus_paid"]].sum()

,group,chest_cnt,wsus_cnt,workshop_use,workshop_paid,wsus_use,wsus_paid
0,A,254055,56784,77509660,66782390,10853200,5538060
1,B,233815,187697,72545280,63921565,34450200,29300955
2,C,105297,178314,34535050,30349475,32436200,28909645
3,D,143708,484478,49845890,43865340,87654600,79067040
4,E,7537,77011,3264090,2876880,13910600,12527690


In [172]:
wsus_and_ws_ratio = wsus_and_ws.groupBy("chest ratio").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

In [194]:
wsus_and_ws_ratio[wsus_and_ws_ratio["chest ratio"] <= 7].sort_values("chest ratio")

,chest ratio,user_cnt
10,0.0,7880
58,1.0,6238
145,2.0,2407
135,3.0,1306
76,4.0,764
183,5.0,611
176,6.0,416
12,7.0,317


In [213]:
wsus_ws_df[wsus_ws_df["chest ratio"] < 10].describe()

,chest_cnt,wsus_cnt,wsus_use,workshop_use,chest ratio
count,20290.000000,20290.000000,20290.000000,20290.000000,20290.000000
mean,36.131345,28.000641,5111.158206,11506.772302,1.416658
std,58.623364,70.020715,12654.639977,22723.311124,1.854158
min,1.000000,1.000000,200.000000,120.000000,0.000000
25%,3.000000,1.000000,200.000000,720.000000,0.000000
50%,13.000000,6.000000,1200.000000,3350.000000,1.000000
75%,44.000000,24.000000,4400.000000,12240.000000,2.000000
max,841.000000,2031.000000,365600.000000,538220.000000,9.000000


In [214]:
wsus_ws_df[wsus_ws_df["chest ratio"] >= 10].describe()

,chest_cnt,wsus_cnt,wsus_use,workshop_use,chest ratio
count,2777.000000,2777.000000,2777.000000,2777.000000,2777.000000
mean,4.071660,99.455167,18022.182211,1348.311127,38.380627
std,8.386785,164.058119,29586.394723,4504.512434,56.309230
min,1.000000,10.000000,1800.000000,120.000000,10.000000
25%,1.000000,20.000000,3600.000000,240.000000,12.000000
50%,1.000000,50.000000,9000.000000,360.000000,20.000000
75%,3.000000,110.000000,19800.000000,1060.000000,40.000000
max,130.000000,2500.000000,450000.000000,124060.000000,780.000000


In [244]:
b_wsus_ws_df = wsus_ws_df[wsus_ws_df["chest ratio"] < 10]
c_wsus_ws_df = wsus_ws_df[wsus_ws_df["chest ratio"] >= 10]

In [245]:
import numpy as np
b_wsus_ws_df['group'] = np.where(b_wsus_ws_df['wsus_cnt']<=3, 'A',
                   np.where(b_wsus_ws_df['wsus_cnt']<=15, 'B',
                   np.where(b_wsus_ws_df['wsus_cnt']<=30, 'C', 
                   np.where(b_wsus_ws_df['wsus_cnt']<=240, 'D', 
                            'E'))))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [246]:
b_wsus_ws_df.groupby("group")["account_id"].count().reset_index()

,group,account_id
0,A,8548
1,B,5119
2,C,2355
3,D,3952
4,E,316


In [247]:
b_wsus_ws_df.groupby("group")["chest ratio"].mean().reset_index()

,group,chest ratio
0,A,0.641788
1,B,1.619262
2,C,1.597028
3,D,2.501771
4,E,4.180380


In [254]:
b_wsus_ws_df.groupby("group")[["wsus_use", "wsus_paid", "workshop_use", "workshop_paid"]].sum().reset_index()

,group,wsus_use,wsus_paid,workshop_use,workshop_paid
0,A,2598000,858205,42718810,36388295
1,B,8310200,5801960,48988930,43131155
2,C,9904600,8453620,33910850,30055460
3,D,57685800,51757470,90624340,80175785
4,E,25206800,22811410,17229480,15234350


In [249]:
import numpy as np
c_wsus_ws_df['group'] = np.where(c_wsus_ws_df['wsus_cnt']<=10, 'A',
                   np.where(c_wsus_ws_df['wsus_cnt']<=50, 'B',
                   np.where(c_wsus_ws_df['wsus_cnt']<=100, 'C', 
                   np.where(c_wsus_ws_df['wsus_cnt']<=800, 'D', 
                            'E'))))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [250]:
c_wsus_ws_df.groupby("group")["account_id"].count().reset_index()

,group,account_id
0,A,229
1,B,1195
2,C,586
3,D,729
4,E,38


In [251]:
c_wsus_ws_df.groupby("group")["chest ratio"].mean().reset_index()

,group,chest ratio
0,A,10.000000
1,B,20.249372
2,C,41.305461
3,D,74.279835
4,E,45.789474


In [253]:
c_wsus_ws_df.groupby("group")[["wsus_use", "wsus_paid", "workshop_use", "workshop_paid"]].sum().reset_index()

,group,wsus_use,wsus_paid,workshop_use,workshop_paid
0,A,416200,282800,74680,32990
1,B,5432000,4549135,439150,191570
2,C,7486400,6678935,441660,225955
3,D,29382800,26387900,1966010,1459220
4,E,7330200,6591860,822760,712380


In [110]:
from pyspark.sql.types import * 

def get_user_info(end_date):
    def classify_country(country_os, country_ip):
        if country_os != 'CN':
            return country_ip
        else:
            return country_os

    country_type_udf = udf(classify_country, StringType())
    meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

    user = load_data_mart("pc", end_date, end_date, "user_master")
    user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
    user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left").withColumnRenamed("accountid", "AccountId").select("AccountId", "pubg_region", "lastlogindate")
    return user

In [111]:
def get_workshop_rate(start_date, end_date):
    user = get_user_info(end_date)
    
    bp_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")) \
        .join(user, "AccountId")
    gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name").like("%workshop%")).withColumnRenamed("account_id", "AccountId") \
        .join(user, "AccountId")
    open_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date) \
        .join(user, "AccountId")
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date) \
        .join(user, "AccountId")
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date) \
        .join(user, "AccountId")
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date) \
        .join(user, "AccountId")
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken") \
        .join(user, "AccountId")
    
    workshop_user = (open_crate.groupBy("pubg_region", "AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
        .unionByName(craft.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
        .unionByName(disassemble.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.groupBy("pubg_region", "AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft"))) \
        .unionByName(gcoin.groupBy("pubg_region", "AccountId").agg(sum("qty").alias("amount")).withColumn("action", lit("gcoin"))) \
        .unionByName(bp_purchase.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("bp_workshop")))
    
    return workshop_user

In [103]:
# 각 day 28
start_date = "2022-06-08"
end_date = "2022-07-05"

ws_start_date = "2022-07-13"
ws_day_28 = "2022-08-09"

In [122]:
get_user_info(ws_day_28).where(col("lastlogindate") >= ws_start_date).select(countDistinct("AccountId").alias("au")).show()

+-------+
|     au|
+-------+
|9660517|
+-------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 43902)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [113]:
ws_user = get_workshop_rate(ws_start_date, ws_day_28)
ws_user.select(countDistinct("AccountId").alias("workshop user count")).show()
ws_user.groupBy("action").agg(countDistinct("AccountId").alias("workshop user count")).orderBy("action").toPandas()

+-------------------+
|workshop user count|
+-------------------+
|            2845464|
+-------------------+



,action,workshop user count
0,bp_workshop,1966678
1,craft,1333668
2,disassemble,1222985
3,gcoin,677314
4,open,2680502
5,repurpose,239705
6,special_craft,270200


In [115]:
ws_user_by_post_seg = ws_user.join(df1.withColumnRenamed("account_id", "AccountId"), "AccountId").groupBy("post_seg").agg(countDistinct("AccountId").alias("workshop user cnt")).orderBy("post_seg").toPandas()
ws_user_by_post_seg_and_action = ws_user.join(df1.withColumnRenamed("account_id", "AccountId"), "AccountId").groupBy("post_seg", "action").agg(countDistinct("AccountId").alias("workshop user cnt")).orderBy("post_seg", "action").toPandas()

In [116]:
ws_user_by_post_seg.to_csv("./ws_user_by_post_seg.csv", index=False)
ws_user_by_post_seg_and_action.to_csv("./ws_user_by_post_seg_and_action.csv", index=False)

In [121]:
ws_user_by_post_seg

,post_seg,workshop user cnt
0,None,188899
1,aA,510300
2,aB,203356
3,aC,25017
4,aD,22680
5,aE,9032
6,bA,88093
7,bB,39731
8,bC,12722
9,bD,21599


In [296]:
special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", ws_start_date, ws_day_28).where(col("Currency") == "artisanstoken")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39317)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39317)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [297]:
special_craft.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- AnalyticEventName: string (nullable = true)
 |-- AnalyticEventType: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- IsOption: boolean (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- LocalPrice: long (nullable = true)
 |-- OptionSalesIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- PlatformProvider: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- Remain: long (nullable = true)
 |-- SalesId: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- TransactionId: string (nullable = true)
 |-- Type: string (nullable = true)



In [298]:
special_craft.select("ItemId").distinct().show(truncate=False)

+-----------------------------------------+
|ItemId                                   |
+-----------------------------------------+
|itemdesc.13000612                        |
|itemdesc.13000611                        |
|itemdesc.14300002,currencydesc.chestkey*1|
+-----------------------------------------+



In [301]:
special_craft_set = special_craft.where(col("ItemId").isin(["itemdesc.13000612", "itemdesc.13000611"])).join(df1.withColumnRenamed("account_id", "AccountId"), "AccountId") \
    .groupBy("seg_group", "group", "seg_group2", "group2").agg(countDistinct("AccountId").alias("user_cnt"), countDistinct("ItemId").alias("craft_cnt")).toPandas()

In [302]:
df1_user_cnt = df1.groupBy("seg_group", "group", "seg_group2", "group2").agg(countDistinct("account_id").alias("total_user_cnt")).toPandas()
pd.merge(df1_user_cnt, special_craft_set, on=["seg_group", "group", "seg_group2", "group2"], how="outer").to_csv("./special_craft_set.csv", index=False)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37602)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [95]:
only_costume_gcoin_by_user3_day28 = spark.createDataFrame(pd.read_csv("./only_costume_gcoin_by_user3_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("wsus_cnt", lit(0)).withColumn("seg_group2", lit("a"))
costume_and_wsus_gcoin_by_user3_day28 = spark.createDataFrame(pd.read_csv("./costume_and_wsus_gcoin_by_user3_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("seg_group2", lit("b"))
only_wsus_gcoin_by_user3_day28 = spark.createDataFrame(pd.read_csv("./only_wsus_gcoin_by_user3_day28.csv")) \
    .withColumnRenamed("group", "group2").withColumn("seg_group2", lit("c")).withColumn("type", lit("wsus")).withColumn("chest_cnt", lit(0))

In [96]:
post_seg28_2 = only_costume_gcoin_by_user3_day28.unionByName(costume_and_wsus_gcoin_by_user3_day28).unionByName(only_wsus_gcoin_by_user3_day28) \
    .withColumnRenamed("paid_use", "paid_use2").withColumnRenamed("total_use", "total_use2") \
    .withColumnRenamed("wsus_cnt", "wsus_cnt2").withColumn("post_seg", concat(col("seg_group2"), col("group2")))

In [97]:
df = pre_seg28.join(post_seg28_2, "account_id", "full_outer")
mysql.insert_table(df, "labs", "workshop_user_seg_df_day28_2")

In [117]:
df2 = mysql.read_table(spark, "labs", "workshop_user_seg_df_day28_2")

In [118]:
ws_user_by_post_seg2 = ws_user.join(df2.withColumnRenamed("account_id", "AccountId"), "AccountId").groupBy("post_seg").agg(countDistinct("AccountId").alias("workshop user cnt")).orderBy("post_seg").toPandas()
ws_user_by_post_seg_and_action2 = ws_user.join(df2.withColumnRenamed("account_id", "AccountId"), "AccountId").groupBy("post_seg", "action").agg(countDistinct("AccountId").alias("workshop user cnt")).orderBy("post_seg", "action").toPandas()

In [119]:
ws_user_by_post_seg2.to_csv("./ws_user_by_post_seg2.csv", index=False)
ws_user_by_post_seg_and_action2.to_csv("./ws_user_by_post_seg_and_action2.csv", index=False)